## Pdf to Markdown

The automatic markdown difficults the extraction of the questions with the titles, as it does not get it right. That's why I disabled the title and keep only the basic md through hdr_info = False

In [12]:
import os
import pymupdf4llm


pdf_name = "500 Data Science Interview Questions.pdf"
md_text = pymupdf4llm.to_markdown(f"../../data/books/{pdf_name}", write_images=True,
                                  image_path=os.path.join("..", "..", "data", "books", pdf_name.split(".")[0]), 
                                  page_chunks=False, hdr_info=False)
md_text

Processing ../../data/books/500 Data Science Interview Questions.pdf...
[                                        ] (0/14[                                        ] (  1/141)

[                                        ] (  2/14[                                        ] (  3/141[=                                       ] (  4/141[=                                       ] (  5/141[=                                       ] (  6/1[=                                       ] (  7/14[==                                      ] (  8/14[==                                      ] (  9/14[==                                      ] ( 10/141[===                                     ] ( 11/1[===                                     ] ( 12/1[===                                     ] ( 13/1[===                                     ] ( 14/14[====                                    ] ( 15/14[====                                    ] ( 16/14[====                                    ] ( 17/141[=====                                   ] ( 18/1[=====                                   ] ( 19/1[=====                                   ] ( 20/1[=====                                   ] ( 21/14[=

'![](../../data/books/500 Data Science Interview Questions/500-Data-Science-Interview-Questions.pdf-0-0.png)\n\n-----\n\n![](../../data/books/500 Data Science Interview Questions/500-Data-Science-Interview-Questions.pdf-1-0.png)\n\n-----\n\n500 Data Science Interview Questions and Answers\n\nVamsee Puligadda\n\n**Disclaimer**\n\nAlthough the author and publisher have made every effort to ensure that the information in this book was correct at press time, the\nauthor and publisher do not assume and hereby disclaim any liability to any party for any loss, damage, or disruption caused by\nerrors or omissions, whether such errors or omissions result from negligence, accident, or any other cause.\n\n**For any suggestions/concerns, please write to us:** indianwolfpublications@gmail.com\n\n**Copyright 2018 Vamsee Puligadda & Indian Wolf Publications**\n\n\n-----\n\n**Foreword**\n\nI am neither the owner of a famous publishing company nor am a top IT company with hundreds of inhouse developers

### Testing and question extraction

In [13]:
import re

question_patterns = r"\*\*Question|\*\*[0-9]{1,3}\)|\*\*\d{1,3}\.|\*\*\[?Q\d{1,3}\.|\[toggle_content|$"
preanswer_patterns = r"\*\*\s*"
answer_patterns = r"\n[^\*][^\*].*?"

# Pattern 1: **Question [0-9]{1,3}. Question text** **Answer:** Answer text
pattern1 = re.compile(rf"\*\*Question ([0-9]{{1,3}})\.?\s*(.+?){preanswer_patterns}\*?\*?Answer:\*?\*?(.*?)(?={question_patterns})", re.DOTALL)

# Pattern 2: **[0-9]{1,3}) Question text** Answer text
pattern2 = re.compile(rf"\*\*([0-9]{{1,3}})\)(.+?){preanswer_patterns}({answer_patterns})(?={question_patterns})", re.DOTALL)

# Pattern 3: **Q?[0-9]{1,3}. Question text** Answer text
pattern3 = re.compile(rf"\*\*Q?(\d{{1,3}})\.\s*(.+?){preanswer_patterns}({answer_patterns})(?={question_patterns})", re.DOTALL)

# Pattern 4: **[Q?[0-9]{1,3}. Question text](Answer image)**
pattern4 = re.compile(rf"\*\*\[Q?(\d{{1,3}})\.\s*(.+?)\]\s*\(({answer_patterns})\)\*\*(?={question_patterns})", re.DOTALL)

# Pattern 5: toggle_content ... Q.[0-9]{1,3} Question text? Answer text
pattern5 = re.compile(rf"toggle_content.*Q?(\d{{1,3}})\.\s*(.+?\?)\s*({answer_patterns})(?={question_patterns})", re.DOTALL)

def extract_question_answer_pairs(markdown_text):
    question_answer_pairs = []

    for pattern in [pattern1, pattern2, pattern3, pattern4, pattern5]:
        match = pattern.findall(markdown_text)
        for id, question, answer in match:
            question = question.replace("**", "")
            question = question.replace("\n", " ")
            answer = answer.replace("\n", " ")
            question_answer_pairs.append((id, question.strip(), answer.strip()))

    return question_answer_pairs

# Example usage
markdown_text = """
**Question 1. What is Python?**
**Answer:** Python is a programming language.
**Question 2. Why use Python?**
**Answer:** It is easy to use.
**3)What are Python libraries?**
Libraries are collections of modules and functions.
**42) What is K-means? How can you select K**
**for K-means?**
**43) What is Collaborative**
** filtering?**
The process of filtering used by most of the recommender systems to find
patterns or information by collaborating viewpoints, various data sources and
multiple agents.
**123. How does Python handle memory?**
Memory management is automatic.
**Q124. What is PEP?**
PEP stands for Python Enhancement Proposal.
**[Q494. What are merge two list and get only unique values?](https://www.besanttechnologies.com/data-science-interview-questions-and-answers)**
**Q455. You have 100 mathletes and 100 math problems. Each mathlete gets**
**[to choose 10 problems to solve. Given data on who got what problem](https://www.besanttechnologies.com/data-science-interview-questions-and-answers)**
**correct, how would you rank the problems in terms of difficulty?**
One way you could do this is by storing a “skill level” for each user and a


-----

“difficulty level” for each problem. We assume that the probability that a user
solves a problem only depends on the skill of the user and the difficulty of the
problem.* Then we maximize the likelihood of the data to find the hidden skill
and difficulty levels.
The Rasch model for dichotomous data takes the form:
{\displaystyle \Pr\{X_{ni}=1\}={\frac {\exp({\beta _{n}}-{\delta _{i}})}
{1+\exp({\beta _{n}}-{\delta _{i}})}},}
where is the ability of person and is the difficulty of item}.
**Q456. You have 5000 people that rank 10 sushis in terms of salt- iness. How**
**[would you aggregate this data to estimate the true saltiness rank in each](https://www.besanttechnologies.com/data-science-interview-questions-and-answers)**
**sushi?**
Some people would take the mean rank of each sushi. If I wanted something
simple, I would use the median, since ranks are (strictly speaking) ordinal and
not interval, so adding them is a bit risque (but people do it all the time and you
probably won’t be far wrong).

[toggle_content title=”Q49. You are AirBnB and you want to test the hypothesis
that a greater number of photographs increases the chances that a buyer selects
the listing. How would you test this hypothesis?
For randomly selected listings with more than 1 pictures, hide 1 random picture
for group A, and show all for group B. Compare the booking rate for the two
groups.
Ask someone for more details.
"""

question_answer_pairs = extract_question_answer_pairs(markdown_text)

# Display the extracted pairs
for (i, question, answer) in question_answer_pairs:
    print(f"Pair {i}:\nQuestion: {question}\nAnswer: {answer}\n")


Pair 1:
Question: What is Python?
Answer: Python is a programming language.

Pair 2:
Question: Why use Python?
Answer: It is easy to use.

Pair 3:
Question: What are Python libraries?
Answer: Libraries are collections of modules and functions.

Pair 42:
Question: What is K-means? How can you select K for K-means? 43) What is Collaborative  filtering?
Answer: The process of filtering used by most of the recommender systems to find patterns or information by collaborating viewpoints, various data sources and multiple agents.

Pair 123:
Question: How does Python handle memory?
Answer: Memory management is automatic.

Pair 124:
Question: What is PEP?
Answer: PEP stands for Python Enhancement Proposal.

Pair 455:
Question: You have 100 mathletes and 100 math problems. Each mathlete gets [to choose 10 problems to solve. Given data on who got what problem](https://www.besanttechnologies.com/data-science-interview-questions-and-answers) correct, how would you rank the problems in terms of diff

<>:36: SyntaxWarning: invalid escape sequence '\d'
<>:36: SyntaxWarning: invalid escape sequence '\d'
C:\Users\a.ramirez.lopez\AppData\Local\Temp\ipykernel_10968\3084383011.py:36: SyntaxWarning: invalid escape sequence '\d'
  markdown_text = """


In [15]:
import json

question_answer_pairs = extract_question_answer_pairs(md_text.replace("\n\n-----\n\n", ""))

book_qa = []
for i, question, answer in question_answer_pairs:
    book_qa.append({"question_n": i,
     "question": question,
     "answer": answer})

with open("../../data/500 Data Science Interview Questions.json", "w") as file:
    json.dump(book_qa, file)

In [124]:
# now work with the markdown text, e.g. store as a UTF8-encoded file
import pathlib
pathlib.Path("output.md").write_bytes(md_text.replace("\n\n-----\n\n", "").encode())

227398

### Guidelines to custom titles in the markdown

In [ ]:
import pymupdf

doc = pymupdf.open(f"../../data/books/{pdf_name}")

for page in doc[4:5]:
    pass

page.get_textpage().extractDICT()['blocks'][1]['lines']

# to hdr_info you can pass a function that gets: page.get_textpage().extractDICT() and returns "", or "#" up to 6

: 

### How to save the markdown

In [ ]:
# now work with the markdown text, e.g. store as a UTF8-encoded file
import pathlib
pathlib.Path("output.md").write_bytes(md_text.encode())